In [2]:
import numpy as np
from skimage import color
from skimage.segmentation import slic
from scipy.spatial.distance import cdist

In [3]:
class SaliencyFilter:
    
    def __init__(self,sigmaU=0.25,sigmaD=20.0,k = 6.0,alpha = 0.033,beta=0.033,nSegments = 300):
        self.__sigmaU = sigmaU
        self.__sigmaD = sigmaD
        self.__k = k
        self.__alpha = alpha
        self.__beta = beta
        self.__nSegments = nSegments
    
    
    def __generateSuperPixels(self,image):
        return slic(image,n_segments=self.__nSegments,enforce_connectivity=True,
                   compactness = 30.0,convert2lab=False)
    
    def __normalize(self,array):
        #change normalization equation
        return (array-array.min())/array.max()-array.min()+1e-13
    
    def __uniqueness(self,superPixels,color,position):
        weight = self.__gaussianWeight(position,self.__sigmaU)
        uniqueness = (cdist(color,color)**2 * weight).sum(axis=1)
        return self.__normalize(uniqueness)
    
    def __distribution(self,superPixels,color,position):
        weight = self.__gaussian_weight(color,self.__sigmaD)
        meanVal = np.dot(weight,position)
        distribution = np.einsum('ij,ji->i',weight,cdist(position,mu)**2)
        return self.__normalize(distribution)
    
    
    